# Intro

## Notes

- All imports necessary for the cell are made in the cell, that way never have to run around running cells in a special order!
 - However! Do need to run the main setup first so the root logger gets set
 
- Also, when running the setup a bunch of times, many loggers seem to get made and the outputs get sent out many times...

# Experiment

## Setup Logger and Other important stuff

In [1]:
# Automatically call debugger on uncaught exception
%pdb on

import os, os.path
from supernn.admin import Log

## Init: Main Setup
log_dir = os.path.join(os.getcwd(), '../data/')
debug_mode = True
generate_log_dir = True
config_path = './config.ini'

args = dict(
    log_dir=log_dir,
    debug_mode=debug_mode,
    gen_log_dir=generate_log_dir,
    config_path=config_path
)

## Init: Set up logger
name = 'error_standards_test'  # Usually set when starting program from command line
prefix = name + '_supernn' if name != '' else supernn

args['name'] = name
args['prefix'] = prefix

log = Log.setup_log_with_args(args)
if not args['debug_mode']:
    log.info('Running in INFO mode')

Automatic pdb calling has been turned ON


22:59:18 [supernn.admin.log   ] [DEBUG   ] : Using tcmalloc!


## Base Runner

In [2]:
import time
import inspect

import supernn.util as util
from supernn.admin import Reporter, Experiment, Log
from supernn.nn import GraphConstructor, Activations; activations = Activations().__dict__
from supernn.datasets import DataInterface

class Runner(Experiment):
    """Parse config and then prepare/run the extracted experiments"""
    
    def __init__(self, config_path, experiment_name):
        """
        Experiment superclass __init__ provides:
            config_manager: Class that interfaces with .ini 
                config file and allows us to get different config parameters
            exp_configs: Config per experiment, obtained by running the 
                parse_experiments_config function which needs to be declared here
        """
        super().__init__(config_path, experiment_name)
        
    # Overriding from Experiment ~Abstract class
    def parse_experiments_config(self):
        """Unpack config"""
        log.info('===Parsing Experiment Config===')
        experiments = []
        
        # Dataset (loaded from corresponding data loader in lib)
        dataset = self.config_manager.get_config('dataset')
        
        # 2D Convolution (1)
        conv2d_1_detail = self.config_manager.get_config('conv2d_1', as_type=int)
        conv2d_1_activation = activations[self.config_manager.get_config('conv2d_1_activation')]
        conv2d_1 = dict(
            x = conv2d_1_detail[0],
            y = conv2d_1_detail[1],
            channels = conv2d_1_detail[2],
            features = conv2d_1_detail[3],
            activation = conv2d_1_activation
        )
        
        # 2D Convolution (2)
        conv2d_2_detail = self.config_manager.get_config('conv2d_2', as_type=int)
        conv2d_2_activation = activations[self.config_manager.get_config('conv2d_2_activation')]
        conv2d_2 = dict(
            x = conv2d_2_detail[0],
            y = conv2d_2_detail[1],
            channels = conv2d_2_detail[2],
            features = conv2d_2_detail[3],
            activation = conv2d_2_activation
        )
        
        # Fully connected Layer
        fcon_1 = dict(
            nodes = self.config_manager.get_config('fcon_1', as_type=int),
            activation = activations[self.config_manager.get_config('fcon_1_activation')]
        )
        
        # Hyperparameters
        learning_rate = self.config_manager.get_config('base_learning_rate', as_type=float)
        batch_size = self.config_manager.get_config('batch_size', as_type=int)
        max_steps = self.config_manager.get_config('max_steps', as_type=int)
        optimiser = self.config_manager.get_config('optimiser')
        
        name = 'conv_lr_{}'.format(learning_rate)
        
        args = dict(
            dataset=dataset,
            conv2d_1=conv2d_1,
            conv2d_2=conv2d_2,
            fcon_1=fcon_1,
            learning_rate=learning_rate,
            batch_size=batch_size,
            max_steps=max_steps,
            optimiser=optimiser,
            name=name
        )
        
        experiments.append(args)
        
        return experiments
    
    def run(self):
        start = time.time()
        for exp_args in self.exp_configs:
            log.debug('Running experiment with args: {}'.format(exp_args))
            self.run_experiment(exp_args)
        log.debug('Finished experiments after %.2fs', time.time() - start)
    
    def run_experiment(self, config):
        start = time.time()
        # prepare savable config
        s_config = {k: (v if not callable(v) else v.__name__) for k, v in config.items()}
        log.debug('experiment config: {}'.format(s_config))
        self.save_subexp_config(s_config, s_config['name'] + '/')
        
        config['data_interface'] = DataInterface(data_name=config['dataset'])
        
        # Set where tensorboard output should go. 
        # Name usually dictated by commandline input,
        # but here it is set in above cell
        Reporter.set_tensorboard_dir(Log.log_dir + 'tensorboard_{}/'.format(config['name']))
        
        # This runs the learner process
        Learner(config).run()
            
        
        log.debug('Finished experiment after %.2fs', time.time() - start)
        

## Graph Runner

In [3]:
from IPython.core.debugger import set_trace

import tensorflow as tf
import numpy as np

from supernn.admin import Reporter, Log; log = Log.get_logger(__name__)
from supernn.nn import InputLayer, FullyConnected, Convolution2D, OutputLayer

## Responsible for taking config and running the experiment
class Learner(object):
    
    def __init__(self, args):
        self.args = args
        self.metrics = None  # tensorflow metrics/ops for model to function
        
        self.graphs = dict(
            tensorflow=None,
            conv2d_graph=None
        )
    
    def run(self):
        # Setup model and graph
        tf_graph = tf.Graph()
        self.graphs['tensorflow'] = tf_graph  # Incase we start using more than one graph.. bcz ideazzzzzzzz
        with tf_graph.as_default():
            self.prepare_data_loader()
            self.build_model()  # sets custom graph
            self.assign_metrics() 
            self.create_and_run_session()
    
    #####
    
    def prepare_data_loader(self):
        self.args['input_data'] = self.args['data_interface'].loader.load_data()
    
    def build_model(self):
        """Test Graph Constructor"""
        log.debug('===Constructing Graph===')
        
        graph = GraphConstructor()
        
        # Data interface, 
        data_interface = self.args['data_interface']
        dataset = data_interface.dataset
        
        
        # Layer Config
        # input_dict
        input_1 = InputLayer(
            dataset['shape'][0], 
            dataset['dtype'], 
            layer_type=dataset['shape_type'], 
            toConv='conv2d',  # conv2d as next layer is a conv2d
            data_interface=data_interface, 
            name='input'
        )
        
        # conv_dict
        conv2d_1 = Convolution2D(
            self.args['conv2d_1']['activation'], 
            self.args['conv2d_1']['x'], 
            self.args['conv2d_1']['y'], 
            self.args['conv2d_1']['channels'], 
            self.args['conv2d_1']['features'], 
            prev_layer='input', 
            name='conv2d_1'
        )
        
        conv2d_2 = Convolution2D(
            self.args['conv2d_2']['activation'], 
            self.args['conv2d_2']['x'], 
            self.args['conv2d_2']['y'], 
            self.args['conv2d_2']['channels'], 
            self.args['conv2d_2']['features'], 
            prev_layer='conv2d_1', 
            name='conv2d_2'
        )
        
        # fully_con_dict
        fCon_1 = FullyConnected(
            self.args['fcon_1']['activation'], 
            self.args['fcon_1']['nodes'], 
            'conv2d_2', 
            name='fcon_1'
        )
        
        # out_dict TODO:: Nothing in config for output... can there be anything useful there?
        out_1 = OutputLayer(
            tf.matmul, 
            10, 
            prev_layer='fcon_1', 
            name='out_1'
        )

        graph.generate_graph_structure(
            properties=[
                ('input', input_1), 
                ('conv2d', conv2d_1), 
                ('conv2d', conv2d_2), 
                ('fully_connected', fCon_1), 
                ('output', out_1)
            ]
        )
    
        self.graphs['conv2d_graph'] = graph
        
    def assign_metrics(self):
        log.info('===Assigning metrics===')
        # Specify metrics
        target_output = tf.placeholder(
            tf.float32, 
            shape=[
                None, 
                self.graphs['conv2d_graph'].model.model_named_layers['out_1'].output.get_shape().as_list()[1]
            ]
        )
        predicted_output = self.graphs['conv2d_graph'].model.model_named_layers['out_1'].output

        with tf.name_scope('cross_entropy'):
            diff = tf.nn.softmax_cross_entropy_with_logits(labels=target_output, logits=predicted_output)
            with tf.name_scope('total'):
                loss = tf.reduce_mean(diff)
        Reporter.report_scalar(loss, 'cross_entropy')

        with tf.name_scope('accuracy'):
            with tf.name_scope('correct_prediction'):
                correct_prediction = tf.equal(tf.argmax(predicted_output, 1), tf.argmax(target_output, 1))
            with tf.name_scope('accuracy'):
                accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        Reporter.report_scalar(accuracy, 'accuracy')

        merger = Reporter.merge_summaries()
        
        train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
        x = self.graphs['conv2d_graph'].model.model_named_layers['input'].input_data
        
        ## Append metrics to experiment arguments (Necessary for session runner)
        self.args['x'] = x
        self.args['train_step'] = train_step
        self.args['merger'] = merger
        self.args['accuracy'] = accuracy
        self.args['target_output'] = target_output
        self.args['predicted_output'] = predicted_output
    
    def create_and_run_session(self):
        log.info('===Preparing Session and Running===')
        
        # For clarity, unwrapping some args
        mnist = self.args['input_data']
        train_step = self.args['train_step']
        merger = self.args['merger']
        target_output = self.args['target_output']
        predicted_output = self.args['predicted_output']
        x = self.args['x']
        accuracy = self.args['accuracy']
        
        # Intialising important variables
        pass_schedule = ['Full', 'Full', 'Full']  # Full pass (Standard batching), Ordered pass (follow new batches)
        prioritised_batch_sets = []
        is_custom_pass = False
        custom_batch = None
        batches_in_epoch = int(mnist.train.num_examples/self.args['batch_size'])
        
        # Helper function - Better in scope for variables access
        def feed_dict(train, custom=False, custom_batch=None):
            """Generate Tensorflow feed_dict: maps data onto Tensor placeholders."""
            if train:
                if custom:
                    # do stuff with custom batch
                    xs, ys = custom_batch
                else:
                    xs, ys = mnist.train.next_batch(self.args['batch_size'])
            else:
                xs, ys = mnist.test.images, mnist.test.labels
            # Load values into Tensorflow variables (accessible in instance args)
            return {x: xs, target_output: ys}
        
        
        with tf.Session() as sess:
            # Initialise all tensorflow variables (otherwise cannot use them)
            sess.run(tf.global_variables_initializer())
            
            # Set up Tensorboard
            train_writer = Reporter.set_writer_path('train', sess.graph)
            test_writer = Reporter.set_writer_path('test')
            
            Reporter.set_saver(tf.train.Saver())
            
            # Main loop
            for epoch in range(len(pass_schedule)):  # Loop while schedules not finished
                
                # Choose which batches to use
                if pass_schedule[epoch] == 'Ordered':
                    log.debug('===Running Ordered Pass===')
                    is_custom_pass = True
                else:
                    log.debug('===Running Standard Pass===')
                    prioritised_batch_sets = []  # Will fill this during pass for ordered run
                    is_custom_pass = False
                    custom_batch = None
                    
                for batch_index in range(batches_in_epoch):
                    # If doing a custom pass, need to use the 
                    # prioritised set of batches generated in full pass
                    if is_custom_pass:
                        # TODO need to create new batches from the prioritised batch sets
                        # Currently just taking from the wrong examples and training on those only
                        custom_batch = prioritised_batch_sets[batch_index][0]
                    
                    # Capture Test results
                    if batch_index % 50 == 0:
                        summary, acc = sess.run([merger, accuracy], feed_dict=feed_dict(False))
                        test_writer.add_summary(summary, batch_index + epoch * batches_in_epoch)
                        log.debug('== Testing - Accuracy at batch step %s: %s =='% (batch_index + epoch * batches_in_epoch, acc))
                    
                    # Training:
                    # Run training and obtain summaries
                    # Also capture the expected vs predicted output,
                    # necessary for new batch creation
                    log.debug('== TRAINING - Mode: {} - Batch: {} - Total Batches: {} =='.format(
                        pass_schedule[epoch], batch_index, batch_index + epoch * batches_in_epoch)
                    )
                    if batch_index % 100 == 99:
                        run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                        run_metadata = tf.RunMetadata()
                        summary, _, target, predicted, samples = sess.run(
                            [merger, train_step, target_output, predicted_output, x], 
                            feed_dict=feed_dict(True, custom=is_custom_pass, custom_batch=custom_batch),
                            options=run_options,
                            run_metadata=run_metadata
                        )
                        train_writer.add_run_metadata(run_metadata, 'step {}'.format(batch_index + epoch * batches_in_epoch))
                        train_writer.add_summary(summary, batch_index + epoch * batches_in_epoch)
                    else:
                        summary, _, target, predicted, samples = sess.run(
                            [merger, train_step, target_output, predicted_output, x],
                            feed_dict=feed_dict(True, custom=is_custom_pass, custom_batch=custom_batch)
                        )
                        train_writer.add_summary(summary, batch_index + epoch * batches_in_epoch)

                    # Check if we are in a Full (standard batching) pass
                    # if yes, then have to create the ordered batches
                    # ready for a prioritised batching pass
                    if not is_custom_pass:
                        # Get samples that failed
                        # TODO: Also capture their feature vectors
                        samples_for_wrong = []
                        labels_for_wrong = []
                        wrong_priority_index = []
                        samples_for_right = []
                        labels_for_right = []
                        right_priority_index = []
                        for res_i in range(len(target)):
                            if tf.argmax(predicted[res_i],1) != tf.argmax(target[res_i], 1):
                                # Input and expected label, associated with the 
                                # output that caused the prediction error.
                                # These are needed for the created batches
                                samples_for_wrong.append(samples[res_i])
                                labels_for_wrong.append(target[res_i])
                                # Later errors are more important, as the model
                                # will be making errors on well optimised values
                                wrong_priority_index.append(
                                    (batches_in_epoch - batch_index)/batches_in_epoch
                                )
                            else:
                                # Also log the successful example, so they
                                # can be used to shuffle into the prioritised batched
                                # e.g. wrong examples later in the program are more
                                # important as it is possible the first errors can be
                                # classified as a result of training. This isn't true 
                                # for the errors after plenty of training.
                                samples_for_right.append(samples[res_i])
                                labels_for_right.append(target[res_i])
                                # Earlier correct predictions are of more interest
                                # as later weights could have alterned the performance on these
                                right_priority_index.append(
                                    1-((batches_in_epoch - batch_index)/batches_in_epoch)
                                )
                        # Pack lists into generators so they can be unpacked later and removed from memory 
#                         gen_wrongens = ((inputs, labels) for inputs in samples_for_wrong for labels in labels_for_wrong)
                        gen_wrongens = (np.array(samples_for_wrong), np.array(labels_for_wrong))
                        #gen_rightens = ((inputs, labels) for inputs in samples_for_right for labels in labels_for_right)
                        gen_rightens = (np.array(samples_for_right), np.array(labels_for_right))

                        prioritised_batch_sets.append(
                            [gen_wrongens, wrong_priority_index, gen_rightens, right_priority_index]
                        )
                
                
                log.debug('== Reached End of Epoch, Number: {} =='.format(epoch))

                # Test progress
                log.debug('== TESTING ==')
                summary, acc = sess.run([merger, accuracy], feed_dict=feed_dict(False))
                test_writer.add_summary(summary, batch_index + epoch * batches_in_epoch)
                log.debug('Test Set -- End of Epoch: {} -- Accuracy: {}'.format(epoch, acc))
            
            log.info('===Reached End of Scheduled Passes===')
            Reporter.save_checkpoint(sess)
            Reporter.save_model_meta()
            
            sess.close

## Main

### Script

In [4]:
runner = Runner(args['config_path'], 'experiment_error_ordering').run()

22:59:19 [supernn.admin.configuration] [DEBUG   ] : Extracting parameters for experiment_error_ordering from path ./config.ini
22:59:19 [supernn.admin.experiment] [DEBUG   ] : Starting experiment with config <Section: experiment_error_ordering>
22:59:19 [__main__            ] [INFO    ] : ===Parsing Experiment Config===
22:59:19 [__main__            ] [DEBUG   ] : Running experiment with args: {'dataset': 'MNIST', 'conv2d_1': {'x': 5, 'y': 5, 'channels': 1, 'features': 32, 'activation': <function relu at 0x7f0ffd4717b8>}, 'conv2d_2': {'x': 5, 'y': 5, 'channels': 32, 'features': 64, 'activation': <function relu at 0x7f0ffd4717b8>}, 'fcon_1': {'nodes': 1024, 'activation': <function relu at 0x7f0ffd4717b8>}, 'learning_rate': 0.01, 'batch_size': 100, 'max_steps': 600, 'optimiser': 'ADAM', 'name': 'conv_lr_0.01'}
22:59:19 [__main__            ] [DEBUG   ] : experiment config: {'dataset': 'MNIST', 'conv2d_1': {'x': 5, 'y': 5, 'channels': 1, 'features': 32, 'activation': <function relu at 0x7

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz


22:59:27 [__main__            ] [DEBUG   ] : ===Constructing Graph===


Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
structure: ('input', <supernn.nn.layers.InputLayer object at 0x7f0ff42a90f0>)
depth: 0
class props: <supernn.nn.layers.InputLayer object at 0x7f0ff42a90f0>
adding input layer
setting input to conv2d shape
structure: ('conv2d', <supernn.nn.layers.Convolution2D object at 0x7f0ff431bcc0>)
depth: 1
class props: <supernn.nn.layers.Convolution2D object at 0x7f0ff431bcc0>
adding conv2d layer
structure: ('conv2d', <supernn.nn.layers.Convolution2D object at 0x7f0ff431b9e8>)
depth: 2
class props: <supernn.nn.layers.Convolution2D object at 0x7f0ff431b9e8>
adding conv2d layer
structure: ('fully_connected', <supernn.nn.layers.FullyConnected object at 0x7f0ff431bda0>)
depth: 3
class props: <supernn.nn.layers.FullyConnected object at 0x7f0ff431bda0>
adding fully connected layer


22:59:27 [__main__            ] [INFO    ] : ===Assigning metrics===
22:59:27 [__main__            ] [INFO    ] : ===Preparing Session and Running===


...

00:48:23 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 180 - Total Batches: 1280 ==
00:48:32 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 181 - Total Batches: 1281 ==
00:48:42 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 182 - Total Batches: 1282 ==
00:48:51 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 183 - Total Batches: 1283 ==
00:49:00 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 184 - Total Batches: 1284 ==
00:49:10 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 185 - Total Batches: 1285 ==
00:49:20 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 186 - Total Batches: 1286 ==
00:49:29 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 187 - Total Batches: 1287 ==
00:49:38 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 188 - Total Batches: 1288 ==
00:49:48 [__main__          

01:00:46 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 255 - Total Batches: 1355 ==
01:00:56 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 256 - Total Batches: 1356 ==
01:01:06 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 257 - Total Batches: 1357 ==
01:01:16 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 258 - Total Batches: 1358 ==
01:01:26 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 259 - Total Batches: 1359 ==
01:01:36 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 260 - Total Batches: 1360 ==
01:01:46 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 261 - Total Batches: 1361 ==
01:01:56 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 262 - Total Batches: 1362 ==
01:02:06 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 263 - Total Batches: 1363 ==
01:02:16 [__main__          

01:13:48 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 330 - Total Batches: 1430 ==
01:13:58 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 331 - Total Batches: 1431 ==
01:14:09 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 332 - Total Batches: 1432 ==
01:14:20 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 333 - Total Batches: 1433 ==
01:14:30 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 334 - Total Batches: 1434 ==
01:14:41 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 335 - Total Batches: 1435 ==
01:14:52 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 336 - Total Batches: 1436 ==
01:15:02 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 337 - Total Batches: 1437 ==
01:15:13 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 338 - Total Batches: 1438 ==
01:15:24 [__main__          

01:27:41 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 405 - Total Batches: 1505 ==
01:27:52 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 406 - Total Batches: 1506 ==
01:28:03 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 407 - Total Batches: 1507 ==
01:28:14 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 408 - Total Batches: 1508 ==
01:28:26 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 409 - Total Batches: 1509 ==
01:28:37 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 410 - Total Batches: 1510 ==
01:28:48 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 411 - Total Batches: 1511 ==
01:28:59 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 412 - Total Batches: 1512 ==
01:29:10 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 413 - Total Batches: 1513 ==
01:29:22 [__main__          

01:42:06 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 480 - Total Batches: 1580 ==
01:42:18 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 481 - Total Batches: 1581 ==
01:42:30 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 482 - Total Batches: 1582 ==
01:42:42 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 483 - Total Batches: 1583 ==
01:42:54 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 484 - Total Batches: 1584 ==
01:43:06 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 485 - Total Batches: 1585 ==
01:43:18 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 486 - Total Batches: 1586 ==
01:43:29 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 487 - Total Batches: 1587 ==
01:43:41 [__main__            ] [DEBUG   ] : == TRAINING - Mode: Full - Batch: 488 - Total Batches: 1588 ==
01:43:53 [__main__          